<a href="https://colab.research.google.com/github/Siu0901/Plain_Project_recommendation_System/blob/main/Plain_%EC%B1%85%EB%A7%88%EB%A3%A8_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U sentence-transformers

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# model = SentenceTransformer("jhgan/ko-sbert-sts") # 옌 한국어임
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
data = pd.read_csv("tmdb_5000_movies.csv")
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [7]:
data['overview'] = data['overview'].fillna('')

In [8]:
genre = data['genres']
genre.head(2)

,genres
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""..."


In [12]:
# 장르 뽑아내기 그지 같은거
genre_list = []
replace_list = ['[','{','"',':',']','}',',','1','2','3','4','5','6','7','8','9','0']
alp_list = ['id','name']

for i in genre:
  g = i
  for j in replace_list:
    g = g.replace(j,'')
  genre_list.append(g)

re_ge = []
for i in range(len(genre_list)):
  re_ge.append([item for item in genre_list[i].replace('name','').replace('id','').split(' ') if item])

re_ge[0]

['Action', 'Adventure', 'Fantasy', 'Science', 'Fiction']

In [13]:
# 자카드 유사도로 추천 후보군 구하기
def jac(list1, list2):
  s1 = set(list1)
  s2 = set(list2)
  union_size = len(s1.union(s2))
  if union_size == 0:
    return 0.0
  return float(len(s1.intersection(s2)) / union_size)

jac_list = []
for i in range(len(re_ge)):
  jac_list.append([])
  for j in range(len(re_ge)):
    jac_list[i].append(jac(re_ge[i],re_ge[j]))

jac_list = np.array(jac_list)

In [14]:
# 각 책에 대한 추천 후보군 구하기
threshold = 0.2 # 대충 유사도 0.2 이상인 애들만
candidates_dict = {}

for i in range(len(jac_list)):
    candidates = [
        j for j, sim in enumerate(jac_list[i])
        if sim >= threshold and j != i
    ]
    candidates_dict[i] = candidates

print(candidates_dict[0])  # 0번 책의 추천 후보군

[1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 59, 61, 62, 63, 64, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 121, 122, 123, 125, 126, 127, 129, 131, 132, 134, 138, 139, 143, 147, 148, 149, 150, 151, 152, 153, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 182, 183, 184, 185, 186, 187, 191, 193, 195, 196, 197, 198, 199, 200, 203, 205, 206, 207, 208, 210, 212, 213, 215, 216, 217, 219, 220, 222, 223, 224, 225, 228, 229, 230, 232, 233, 234, 235, 237, 238, 239, 240, 241, 242, 243, 245, 249, 254, 256, 257, 259, 260, 262, 263, 266, 267, 270, 271, 274, 275, 276, 277, 278, 279, 289, 290, 291, 292, 294, 296, 299, 300, 301, 302, 304, 305, 306, 307, 309, 310, 311, 312, 31

In [17]:
# 후보 영화 내용에 대한 코사인 유사도 계산 함수
def Cosine(n):
  doc_input = data[data['title'] == n]['overview'].iloc[0]
  movie_index = data[data['title'] == n].index[0]

# 후보군 리스트
  valid_candidate_indices = [idx for idx in candidates_dict[movie_index] if idx in data.index]
  doc_candidates = data['overview'].loc[valid_candidate_indices].tolist()

# 임베딩
  embedding_input = model.encode([doc_input])
  embeddings_candidates = model.encode(doc_candidates)

# 코사인 유사도 비교 - 입력 영화과 후보군들
  similarities = cosine_similarity(embedding_input, embeddings_candidates)[0]
  return similarities

In [18]:
# 후보 영화 장르에 대한 자카드 유사도 함수
def main_jac(n):
  movie_index = data[data['title'] == n].index[0]

  can_gen = candidates_dict[movie_index]
  movie_gen = re_ge[movie_index]

  jac_sim = []
  for i in can_gen:
    jac_sim.append(jac(movie_gen,re_ge[i]))

  return np.array(jac_sim)

In [86]:
# 진짜 ㄱ같은 전체 영화 제작사 뽑아내기
comapnies = data['production_companies']
replace_list2 = ['[','{','"',':',']','}','1','2','3','4','5','6','7','8','9','0']

comapnies_list = []
for i in comapnies:
  c = i
  for j in replace_list2:
    c = c.replace(j,'')
  comapnies_list.append(c)

re_co = []
for i in range(len(comapnies_list)):
  re_co.append([item for item in comapnies_list[i].replace('name','').replace('id','').split(',') if item])

real_co = []
for i in range(len(re_co)):
  filtered_row = [item.strip() for item in re_co[i] if item.strip()]
  real_co.append(filtered_row)

filtered_co = []
for i in range(len(real_co)):
  filtered_co.append([])
  for j in range(len(real_co[i])):
    filtered_co[i].append(real_co[i][j].strip())

print(filtered_co[0]) # 0번 영화의 제작사들

['Ingenious Film Partners', 'Twentieth Century Fox Film Corporation', 'Dune Entertainment', 'Lightstorm Entertainment']


In [87]:
# 후보 영화 제작사에 대한 자카드 유사도 함수
def comapnies_jac(n):
  movie_index = data[data['title'] == n].index[0]
  movie_co = filtered_co[movie_index]

  # 영화 후보군 제작사 추리기
  can_co = []
  for i in candidates_dict[movie_index]:
    can_co.append(filtered_co[i])

  # 제작사 자카드 유사도 계산
  comapnies_sim = []
  for i in can_co:
    comapnies_sim.append(jac(movie_co,i))

  return np.array(comapnies_sim)

In [88]:
# 최종 추천 후보군 반환 함수
def recommendation(n):
  Final_Score = 0.6 * Cosine(n) + 0.3 * main_jac(n) + 0.1 * comapnies_jac(n)

  idx = zip(candidates_dict, Final_Score)
  idx = sorted(idx, key=lambda x: x[1], reverse=True)
  movie_idx = idx[:20]

  recommendation_movies = []
  for i in movie_idx:
    movie = data['title'][i[0]]
    recommendation_movies.append(movie)

  return recommendation_movies

In [89]:
# Pirates of the Caribbean: The Curse of the Black Pearl
n = input("영화 제목을 입력하세요: ")
recommendation(n)

영화 제목을 입력하세요: Avatar


['Meet the Fockers',
 'Dreamgirls',
 'Bicentennial Man',
 'TRON: Legacy',
 'The Road to El Dorado',
 'Flushed Away',
 'Star Trek Into Darkness',
 'Epic',
 'Evan Almighty',
 'Quantum of Solace',
 'Captain America: The First Avenger',
 'Rango',
 'Terminator Salvation',
 'The Golden Compass',
 'Jack Ryan: Shadow Recruit',
 'Bad Company',
 'Osmosis Jones',
 'Speed 2: Cruise Control',
 'Jack the Giant Slayer',
 'Non-Stop']